In [1]:
from py2neo import Graph,Node,walk,Relationship,NodeSelector
import pandas as pd

In [66]:
#英雄列表
hero=pd.read_csv('./data/hero.csv',header=0,encoding='gbk')['name']

#装备列表
weapon=pd.read_excel('./data/data_modified.xlsx',header=0)['物品名称']

nodes=list(hero)+list(weapon)

#关系列表
relation=[u'相似',u'克制',u'搭配',u'推荐',u'适合用于对抗',u'同类互斥']

#英雄属性列表
HP_all=['HP',u'生命值',u'血量',u'血上限',u'红',u"血"]
MP_all=['MP',u'法力值',u'蓝量',u'蓝']
HP_recover_all=[u'HP_recover',u'每5秒回血','回血']
MP_recover_all=[u'MP_recover',u'每5秒回复法力值',u'每5秒回蓝',u'回蓝']
R_cooling_all=['R_cooling',u'大招冷却时间']
R_cost_all=['R_cost',u'大招消耗']
skill_R_all=['R',u'大招']
attack_all=['attack',u'物理攻击',u'攻击力',u'攻击']
attack_range_all=['attack_range',u'攻击距离',u'近战',u'远程']
aa=['defense',u'护甲',u'物理防御']
bb=['skill_passive',u'被动技能',u'被动']
cc=['skill_1',u'一技能',u'1技能',u'技能一',u'技能1']
dd=['skill_2',u'二技能',u'2技能',u'技能二',u'技能2']
ee=['skill_1_cooling',u'1技能冷却时间',u'技能1冷却时间',u'一技能冷却时间',u'技能一冷却时间']
ff=['skill_2_cooling',u'2技能冷却时间',u'技能2冷却时间',u'二技能冷却时间',u'技能二冷却时间']
gg=['skill_1_cost',u'1技能消耗',u'技能1消耗',u'一技能消耗',u'技能一消耗']
hh=['skill_2_cost',u'2技能消耗',u'技能2消耗',u'二技能消耗',u'技能二消耗']
ii=['speed',u'移动速度',u'移速',u'速度']
jj=['tag',u'类型']
hero_property=[aa,bb,cc,dd,ee,ff,gg,hh,ii,jj,HP_all,MP_all,HP_recover_all,MP_recover_all,R_cooling_all,R_cost_all,skill_R_all,attack_all,attack_range_all]
        
#道具属性同义词库
wsx='property	法术吸血	物理吸血	暴击率	法术防御	物理防御	每5秒回蓝	每5秒回血	最大法力	最大生命	减cd	攻击速度	移速	法术攻击	物理攻击'.split('	')

wsx.append(u'被动')
wsx.append(u'主动') 

synonym=hero_property+[wsx] #同义词表

#属性词表
propertys=[s[0] for s in synonym ]

In [3]:
synonym

[['defense', '护甲', '物理防御'],
 ['skill_passive', '被动技能', '被动'],
 ['skill_1', '一技能', '1技能', '技能一', '技能1'],
 ['skill_2', '二技能', '2技能', '技能二', '技能2'],
 ['skill_1_cooling', '1技能冷却时间', '技能1冷却时间', '一技能冷却时间', '技能一冷却时间'],
 ['skill_2_cooling', '2技能冷却时间', '技能2冷却时间', '二技能冷却时间', '技能二冷却时间'],
 ['skill_1_cost', '1技能消耗', '技能1消耗', '一技能消耗', '技能一消耗'],
 ['skill_2_cost', '2技能消耗', '技能2消耗', '二技能消耗', '技能二消耗'],
 ['speed', '移动速度', '移速', '速度'],
 ['tag', '类型'],
 ['HP', '生命值', '血量', '血上限', '红'],
 ['MP', '法力值', '蓝量', '蓝'],
 ['HP_recover', '每5秒回血', '回血'],
 ['MP_recover', '每5秒回复法力值', '每5秒回蓝', '回蓝'],
 ['R_cooling', '大招冷却时间'],
 ['R_cost', '大招消耗'],
 ['R', '大招'],
 ['attack', '物理攻击', '攻击力', '攻击'],
 ['attack_range', '攻击距离', '近战', '远程'],
 ['property',
  '法术吸血',
  '物理吸血',
  '暴击率',
  '法术防御',
  '物理防御',
  '每5秒回蓝',
  '每5秒回血',
  '最大法力',
  '最大生命',
  '减cd',
  '攻击速度',
  '移速',
  '法术攻击',
  '物理攻击',
  '被动',
  '主动']]

In [51]:
graph=Graph("http://localhost:7474",password="pass") 

In [52]:
def find_node(graph,node_name,output=None):
    s=NodeSelector(graph)  
    try:
        if(output):
            return output(s.select(name=node_name).first())
        else:
            return s.select(name=node_name).first()
    except:
        return '没有这个'+node_name+'节点，查询失败'

find_node(graph,'白起',dict)

{'skill_2_cooling': '14/13.4/12.8/12.2/11.6/11',
 'attack_range': '近程',
 'MP': 0,
 'skill_2_cost': 60,
 'skill_passive': '被动：每次击败英雄或助攻会回复生命值，击杀英雄回复最大生命6%，击败小兵野怪回复最大生命1%',
 'skill_1_cooling': '10/9/8/7/6/5',
 'HP': 3510,
 'R_cost': 80,
 'HP_recover': 53,
 'speed': 390.0,
 'R_cooling': '30/25/20',
 'skill_1_cost': 40,
 'R': '白起嘲讽范围内的敌军持续2秒，期间自身受到的伤害将被降低20%/25%/30%；被动：受到攻击每层会提升移动速度10点与护甲30/60/90点，最多叠加3层持续3秒',
 'defense': 120,
 'attack': 158,
 'magic_defense': 50,
 'MP_recover': 0,
 'name': '白起',
 'skill_2': '白起挥出长镰，造成150/185/220/255/290/325（+70%物理加成）点物理伤害并将面前的敌人拉回到身前造成减速',
 'tag': ' 突进  坦克 ',
 'skill_1': '白起回旋大镰血洗四方，对范围内的敌人造成125/150/175/200/225/250（+120%物理加成）点物理伤害；被动，白起受伤时有几率释放血之回响'}

In [32]:
#查询node的属性
def find_node_property(graph,node_name,props):
    tmp=find_node(graph,node_name,output=dict)
    if(type(props)==list):
        result={}
        for i in props:
            try:
                result[i]=tmp[i]
            except:
                print('输入的属性'+i+'有误，没有这个属性')
        return result
    else:
        return tmp[props]

In [70]:
find_node_property(graph,'赵云',['skill_1','skill_2'])
find_node_property(graph,'赵云','attack_range')

3267

In [34]:
#查询与节点有特定关系的节点
def find_node_from_rela(graph,node_name,relationship):
    node=find_node(graph,node_name)
    tmp=[]
    try:        
        for n in graph.match(start_node=node,rel_type=relationship):
            tmp.append(n.end_node()["name"])
        return tmp
    except:
        return '输入有误，查询失败'

In [35]:
find_node_from_rela(graph,'赵云','克制')

['高渐离', '姜子牙', '鲁班七号', '孙尚香', '王昭君', '武则天', '嬴政']

In [36]:
#查询两个节点的关系
def find_rela(graph,node_name_1,node_name_2):
    node_1=find_node(graph,node_name_1)
    node_2=find_node(graph,node_name_2)
    try:
        rel=graph.match_one(start_node=node_1,end_node=node_2)
        return node_name_1+rel.type()+node_name_2
    except:
        try:
            rel=graph.match_one(start_node=node_2,end_node=node_1)
            return node_name_2+rel.type()+node_name_1
        except:
            return node_name_1+'与'+node_name_2+'没有关系'

In [39]:
find_rela(graph,'赵云','王昭君')  
# find_rela(graph,'王昭君','赵云') 
# find_rela(graph,'赵云','曹操') 


'赵云克制王昭君'

In [40]:
#查询两个node之间是否存在特定关系
def is_node_rela(graph,node_name_1,node_name_2,rela):
    tmp=find_rela(graph,node_name_1,node_name_2)
    return rela in tmp

is_node_rela(graph,'赵云','大乔','搭配')  

True

In [41]:
#查询node的某个属性值是否为指定值
def is_node_property(graph,node_name,prop,value):    
    return value==find_node_property(graph,node_name,prop)
 
is_node_property(graph,'赵云','attack_range','近程')
is_node_property(graph,'赵云','attack_range','远程')


False

# 查询语句的处理

In [54]:
def get_node(s,nodes):
    node=[h for h in nodes if h in s]
    return node
    
get_node('赵云的基本属性',nodes)
get_node('赵云和曹操谁的攻击力比较强',nodes)

def get_rela(s,relationship):
    rela=[r for r in relationship if r in s]
    return rela

get_rela('赵云克制谁',relation)

def get_property(s,propertys):
    prop=[p for p in propertys if p in s]
    return prop

get_property('赵云和曹操谁的攻击力比较强',propertys)
get_property('赵云和曹操谁的attack比较强',propertys)

['attack']

# 同义词替换

In [56]:
def synonym_replace(question,synonym):
    replace_word=[]
    for words in synonym:
        for i in words:
            if(i in question):
                question=question.replace(i,words[0])
                replace_word.append(i)
    return question,replace_word

synonym_replace('赵云的攻击力是多少',synonym)
# synonym_replace('铁剑加多少暴击率',synonym)

('赵云的attack是多少', ['攻击力'])

# 查询模板编写

In [44]:
def search(s,syn,nodes,relation,propertys,graph,output=None):
    node=get_node(s,nodes)
    rela=get_rela(s,relation)
    prop=get_property(s,propertys)
    n=len(node)
    r=len(rela)
    p=len(prop)
    if(all([n==1,r==0,p==0])):
        return find_node(graph,node[0],output=output)
    elif(all([n==1,r==0,p!=0])):
        if(prop[0]=='property'):
            tmp=find_node_property(graph,node[0],prop)['property'].split(' ')
            for t in tmp:
                res=[f for f in syn if f in t]
                if(res):
                    return res
                else:
                    return node[0]+'没有这个属性'                            
        else:
            return find_node_property(graph,node[0],prop)
    elif(all([n==1,r==1,p==0])):
        return find_node_from_rela(graph,node[0],rela[0])
    elif(all([n==2,r==0,p==0])):
        return find_rela(graph,node[0],node[1])
    elif(all([n==2,r==1,p==0])):
        return is_node_rela(graph,node[0],node[1],rela[0])
    else:
        return '查询的问题太复杂，重新输入'                

In [80]:
search('赵云和曹操','',nodes,relation,propertys,graph,output=None)        
search('赵云的skill_1','',nodes,relation,propertys,graph,output=None)

question="曹操大招冷却时间"
s,syn=synonym_replace(question,synonym)
search(s,syn,nodes,relation,propertys,graph,output=None)

{'R_cooling': '1/30/00',
 'R': '曹操以鲜血意志强化大剑，开启时对附近敌人造成200/250/300（+72%物理加成）点物理伤害，并大幅度提升自身攻击力80及20%韧性效果，每次普攻或技能命中敌方将回复自身生命120/180/240（+40%物理加成）点，持续8秒'}